In [9]:
import polars as pl
import os
from datetime import datetime

In [10]:
NOTIFICATIONS_TILES_PATH = '../data/transformed/notifications_with_tiles.csv'
USERS_PATH = '../data/raw/hub_users.json'
FINAL_OUTPUT_PATH = '../data/transformed/notifications_users.csv'

df_notifications_tiles = pl.read_csv(NOTIFICATIONS_TILES_PATH)
# print(f"Loaded notifications with tiles: {df_notifications_tiles.shape[0]} records with {df_notifications_tiles.shape[1]} columns")
# df_notifications_tiles.head()

In [11]:
# Load user data
df_users = pl.read_json(USERS_PATH)
# print(f"Loaded users data: {df_users.shape[0]} records with {df_users.shape[1]} columns")

# Extract required columns from users data
df_users_selected = df_users.select(
    pl.col('capstone_email'),
    pl.col('jobtitle'),
    pl.col('authorization').struct.field('role').alias('authorization_role')
)

# print("Selected user columns:")
# df_users_selected.head()

## Transformation

In [13]:
# Join notifications_tiles with users based on role
# Ensure the join columns have the same name
df_notifications_users = df_notifications_tiles.join(
    df_users_selected,
    left_on='tile_roles',
    right_on='authorization_role',
    how='inner'
)

# print(f"Joined notifications with users: {df_notifications_users.shape[0]} records with {df_notifications_users.shape[1]} columns")
# df_notifications_users.head()

In [14]:
# Write the joined data to CSV
os.makedirs(os.path.dirname(FINAL_OUTPUT_PATH), exist_ok=True)
df_notifications_users.write_csv(FINAL_OUTPUT_PATH)
print(f'Notifications with users saved to {FINAL_OUTPUT_PATH}')

Notifications with users saved to ../data/transformed/notifications_users.csv
